# MAE


ViTMAE 模型由 Meta 的 FAIR 团队的 Kaiming He、Xinlei Chen、Saining Xie、Yanghao Li、Piotr Dollár 和Ross Girshick 在 [Masked Autoencoders Are Scalable Vision Learners](https://arxiv.org/abs/2111.06377v2) 中提出。该论文表明，通过对视觉Transformer (ViT) 进行预训练，使其能够重建被遮挡图像块的像素值，经过微调后，模型的效果优于监督预训练。

论文的摘要如下：

> 本文展示了掩码自动编码器（MAE）作为计算机视觉领域可扩展的自监督学习方法。我们的MAE方法非常简单：我们对输入图像中的随机图像块进行掩码，然后重建缺失的像素。此方法基于两个核心设计。首先，我们开发了一种不对称的编码器-解码器架构，编码器仅处理可见的图像块（无需掩码标记），同时使用轻量化的解码器从潜在表示和掩码标记重建原始图像。其次，我们发现对输入图像进行高比例的掩码（例如75%）可以形成一个具有挑战性且有意义的自监督任务。结合这两种设计，我们能够高效地训练大模型：训练速度加快了3倍或更多，并且精度得到提升。我们的可扩展方法能够学习出具有良好泛化能力的高容量模型，例如，一个原生的ViT-Huge模型在仅使用 ImageNet-1K 数据的情况下达到了最佳精度（87.8%）。在下游任务中的迁移性能优于监督预训练，并显示出良好的扩展行为。

![](../images/mae_2024-10-26-17-42-45.png)


官方的 Pytorch 实现代码（github） ：https://github.com/facebookresearch/mae

# Patch Embedding

整个模型计算过程的第一步是 Patch Embedding，也就是将 2D 的图像格式转为一个 Patch 化的序列输入格式。

* 输入的形状是：`[batch_size, channels, height, width]`
* 输出的形状是：`[batch_size, num_patches, dim]`

该操作是 ViT 中引入的一个经典操作，我们可以用 Conv2D 来实现，也可以通过一个 MLP 来实现。

In [1]:
import torch
from torch import nn

batch_size = 1
img_size=224
patch_size=16
in_chans=3
embed_dim=1024

img = torch.randn(batch_size, in_chans, img_size, img_size)

proj = nn.Conv2d(in_chans, embed_dim, patch_size, patch_size)
x = proj(img) # batch_size, embed_dim, h_patches, w_patches
x = x.flatten(2).transpose(1, 2)  # NCHW -> NLC
print(x.shape)

torch.Size([1, 196, 1024])


我们可以通过 `timm` 中实现的 `PatchEmbed` 来验证我们的实现，由于 `nn.Conv`的初始化随机权重不一致，导致结果无法对齐。

In [2]:
from timm.models.vision_transformer import PatchEmbed
patch_embed = PatchEmbed(img_size, patch_size, in_chans, embed_dim)
x1 = patch_embed(img)
print(x1.shape)

torch.Size([1, 196, 1024])


# 2D Position Embedding

在 MAE 的 Encoder 和 Decoder 中，我们都需要对输入的 Patch 序列添加位置编码的信息，这里区别于 NLP 中的一维位置编码，这里需要添加二维位置编码。

In [3]:
import numpy as np

def get_2d_sincos_pos_embed(embed_dim, grid_size, cls_token=False):
    """
    grid_size: int of the grid height and width
    return:
    pos_embed: [grid_size*grid_size, embed_dim] or [1+grid_size*grid_size, embed_dim] (w/ or w/o cls_token)
    """
    # grid_h 代表整个 grid 上每个 patch 的高度序号
    grid_h = np.arange(grid_size, dtype=np.float32)
    # grid_w 代表整个 grid 上每个 patch 的宽度序列
    grid_w = np.arange(grid_size, dtype=np.float32)
    grid = np.meshgrid(grid_w, grid_h)  # here w goes first
    grid = np.stack(grid, axis=0) # (2, grid_size, grid_size)

    grid = grid.reshape([2, 1, grid_size, grid_size]) # [2, 1, grid_size, grid_size]
    pos_embed = get_2d_sincos_pos_embed_from_grid(embed_dim, grid)
    if cls_token:
        pos_embed = np.concatenate([np.zeros([1, embed_dim]), pos_embed], axis=0)
    return pos_embed


def get_2d_sincos_pos_embed_from_grid(embed_dim, grid):
    assert embed_dim % 2 == 0

    # grid[0]是 grid 中每个 patch 的 水平方向上的位置 [[0,1,2,3...],[0,1,2,3...]]
    # grid[1]是 grid 中每个 patch 的 垂直方向上的位置 [[0,0,0,0...],[1,1,1,1...]]
    # use half of dimensions to encode grid_h
    emb_h = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[0])  # (H*W, D/2)
    emb_w = get_1d_sincos_pos_embed_from_grid(embed_dim // 2, grid[1])  # (H*W, D/2)

    emb = np.concatenate([emb_h, emb_w], axis=1) # (H*W, D)
    return emb


def get_1d_sincos_pos_embed_from_grid(embed_dim, pos):
    """
    embed_dim: output dimension for each position
    pos: a list of positions to be encoded: size (M,)
    out: (M, D)
    """
    assert embed_dim % 2 == 0
    omega = np.arange(embed_dim // 2, dtype=np.float32)
    omega /= embed_dim / 2.
    omega = 1. / 10000**omega  # (D/2,)

    pos = pos.reshape(-1)  # (M,)
    out = np.einsum('m,d->md', pos, omega)  # (M, D/2), outer product

    emb_sin = np.sin(out) # (M, D/2)
    emb_cos = np.cos(out) # (M, D/2)

    emb = np.concatenate([emb_sin, emb_cos], axis=1)  # (M, D)
    return emb

In [4]:
num_patches = x.size(1)
print(f"num_patches = {num_patches}")
grid_size = int(num_patches**.5)
pos_embed_npy = get_2d_sincos_pos_embed(embed_dim, grid_size, cls_token=True)
pos_embed = nn.Parameter(torch.from_numpy(pos_embed_npy).float().unsqueeze(0))
# 考虑上 CLS Token，所以序列长度为 num_patches + 1
print(pos_embed.shape)

num_patches = 196
torch.Size([1, 197, 1024])


在 `x` 上添加位置编码的信息：

In [5]:
x = x + pos_embed[:, 1:, :]

# Random Masking


MAE 模型对输入图像 Patches 进行随机掩码后，输入给 Encoder 的只是没有被 Mask 的部分，而对于 Decoder，它的输入除了 Encoder 的输入外，还需要拼接上 Mask 的 Patch Embedding，然后把它们恢复回原来的图像 Patch 的顺序。

![](../images/random_masking.drawio.svg)

In [6]:
import torch

mask_ratio = 0.75 # 掩码比例
N, L, D = x.shape # 1, 7, 8
len_keep = int(L * (1 - mask_ratio)) # 10 * 0.25 = 2

noise = torch.rand(N, L) # noise in [0, 1]

ids_shuffle = torch.argsort(noise, dim=1)
ids_restore = torch.argsort(ids_shuffle, dim=1)

ids_keep = ids_shuffle[:, :len_keep]
x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D)) # batch_size, len_keep, embed_dim
print(f"x_masked shape: {x_masked.shape}")

mask = torch.ones(N, L)
mask[:, :len_keep] = 0
mask = torch.gather(mask, dim=1, index=ids_restore)

x_masked shape: torch.Size([1, 49, 1024])


# 添加 CLS Token

In [7]:
cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
cls_token = cls_token + pos_embed[:, :1, :] # 为 CLS Token 添加位置编码
cls_token = cls_token.expand(x.size(0), -1, -1) # 在 batch 的维度上进行复制

x = torch.concat([x, cls_token], dim=1) # 在长度维度上拼接在一起

# Encoder Transformer

在对整个图像进行 Patch Embedding 和进行 Random Mask 后，现在我们将 `x` 输入到一个标准的 Transformer Encoder 中。

In [8]:
nheads = 16
num_encoder_layers = 24
encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=16, dim_feedforward=4 * embed_dim, batch_first=True)
encoder = nn.TransformerEncoder(encoder_layer, num_encoder_layers)

latent = encoder(x_masked)
print(f"latent tensor shape: {latent.shape}")

latent tensor shape: torch.Size([1, 49, 1024])


# Decoder Embed

在进行 Decoder 之前，由于 decoder 的维度可能和 Encoder 的维度不一致，所以这里有一个 MLP 层进行维度的转换。

In [9]:
decoder_embed_dim=512
decoder_embed = nn.Linear(embed_dim, decoder_embed_dim, bias=True)
x = decoder_embed(latent)
print(x.shape)

torch.Size([1, 49, 512])


# Restore Patches

区别与 Encoder，Decoder 的输入是包括了被 Mask 掉的 Patch 的部分的，只是这部分输入的是一个随机初始化的 masked_token，把我们需要将 Encoder 输出的 Latent 和 masked_token 拼在一起后，再还原为原来的 Patch 的位置关系。


In [10]:
mask_token = nn.Parameter(torch.zeros(1, 1, decoder_embed_dim))
mask_tokens = mask_token.repeat(x.size(0), ids_restore.size(1) - x.size(1) + 1, 1)
x_ = torch.concat([x[:, 1:, :], mask_tokens], dim=1)
x_ = torch.gather(x_, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, x_.size(2)))
x = torch.concat([x[:,:1,:], x_], dim = 1) # 把 CLS Token 再加回来
print(x.shape)

torch.Size([1, 197, 512])


# 添加 Decoder 位置编码

In [11]:
pos_embed_npy = get_2d_sincos_pos_embed(decoder_embed_dim, grid_size, cls_token=True)
decoder_pos_embed = nn.Parameter(torch.from_numpy(pos_embed_npy).float().unsqueeze(0))

x = x + decoder_pos_embed

# Decoder Transformer

在 MAE 中， Decoder Transformer 相较于 Encoder 一般来说会比较轻量。虽然说是 Decoder，实现上来是双向注意力。

In [12]:
decoder_depth = 8
decoder_num_heads=16

decoder_layer = nn.TransformerEncoderLayer(d_model=decoder_embed_dim, nhead=decoder_num_heads, dim_feedforward=4 * embed_dim, batch_first=True)
decoder = nn.TransformerEncoder(decoder_layer, decoder_depth)

pred = decoder(x)
print(f"pred tensor shape: {pred.shape}")

pred tensor shape: torch.Size([1, 197, 512])


在经过 Decoder 的多层 Transformer Block 处理后，一般会有一个 Decoder Head，将每个 patch 的特征维度再转换为 patch_size * patch_size * in_channs

同时我们也在最后去除掉 CLS Token

In [13]:
decoder_head = nn.Linear(decoder_embed_dim, patch_size * patch_size * in_chans)
pred = decoder_head(pred)
pred = pred[:, 1:, :]
print(f"final pred tensor shape: {pred.shape}")

final pred tensor shape: torch.Size([1, 196, 768])


# 计算损失

在 MAE 的损失函数设计中，我们希望 Decoder 对于 Mask掉的 Patch 的像素值能够回归预测出来，所以我们使用的是 MSELoss

我们先计算每个 patch 的 MSE，然后再把那些非 mask 的 patch 部分的 loss 给 mask 掉，最后计算总和。

In [14]:
target = img.reshape(img.shape[0], img.shape[1], img.shape[2] // patch_size, patch_size, img.shape[3] // patch_size, patch_size)
# N,C, H, P, W, P -> N,H,W,P,P,C
target = target.permute(0,2,4,3,5,1).reshape(img.shape[0], -1, patch_size * patch_size * in_chans)
print(target.shape)

torch.Size([1, 196, 768])


In [15]:
loss = (pred - target) ** 2
loss = loss.mean(dim=-1)  # [N, L], mean loss per patch
loss = (loss * mask).sum() / mask.sum() # mean loss on removed patches